# Импорт библиотек и обучающих данных:


---

In [1]:
import numpy as np
import pandas as pd
import zipfile
from matplotlib import pyplot as plt
import shutil 
from tqdm import tqdm
import torch
import torchvision
import time
import copy
from torchvision import transforms, models
import os
import torch.nn as nn

In [2]:
torch.cuda.is_available()

True

# Разархивация изображений

---

In [3]:
# with zipfile.ZipFile(r"data\welds.zip",'r') as zip_obj:
#         zip_obj.extractall(r"data\binary_root")
# data_root = r"data\binary_root"
# print('Файлы выгрузились из архива успешно!') 

# Разметка данных:

---

У меня есть json файл с разметкой. Надо его пропарсить. Добавляю столбец для класса в бинарной классификации

In [4]:
# js = os.path.join(data_root, r"al5083\train\train.json")
# labels = pd.read_json(js, typ='series')
# labels = labels.to_frame().reset_index().rename(columns={'index':'path',0:'class'})
# def create_binary_label(row):
#     if row['class'] == 0:
#         return 'good_weld'
#     else:
#         return 'bad_weld'
# labels['binary'] = labels.apply(create_binary_label, axis=1)
# labels = labels.sort_values(by='binary').reset_index().drop(['index','class'],axis=1)
# classes = labels['binary'].unique()
# labels

Для повышения точности модели решено создать две нейросети - первая будет бинарно классифицировать хорошие и плохие швы, а вторая будет определять вид дефекта у плохих швов. Для балансировки данных в бинарной классификации необходимо удвоить количество хороших швов за счёт аугментации.

# Разбивка данных на обучающие и валидационные:

---



Собираю вместе все good и bad weld's

In [ ]:
# class_names = classes
# train_data = os.path.join(data_root, r'al5083\train')
# for class_name in class_names: 
#     for i, file_name in enumerate(tqdm(labels['path'].loc[labels['binary']==class_name].tolist())):
#         pic_name = str(class_name) + '_' + str(i) + '.png'
#         shutil.copy(os.path.join(train_data, file_name), os.path.join(os.path.join(data_root, r"welds",class_name, pic_name)))

Отзеркаливаю good welds

In [ ]:
# import os
# from PIL import Image

# # Путь к директории с изображениями
# image_dir = r"C:\Users\HP\Documents\Artificial_Intelligence\Проекты для CV\weld_inspection\data\binary_root\welds\good_weld"

# # Получение списка файлов изображений в директории
# image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f)) and f.endswith(('.jpg', '.jpeg', '.png'))]

# # Создание зеркальных копий изображений
# for file_name in tqdm(image_files):
#     file_path = os.path.join(image_dir, file_name)
#     image = Image.open(file_path)
#     flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)
#     flipped_file_name = "flipped_" + file_name
#     flipped_file_path = os.path.join(image_dir, flipped_file_name)
#     flipped_image.save(flipped_file_path)

# print("Зеркальные копии изображений успешно созданы.")

Создаю валидационную и обучающую папки и загружаю в них данные:

In [ ]:
# # train_dir = os.path.join(data_root,'train')
# # val_dir = os.path.join(data_root,'val')
# # class_names = classes # некий список классов нашей классификации.

# # for dir_name in [train_dir, val_dir]: #итерация через 2 строчки
# #     for class_name in class_names: # для каждого имени класса  в списке классов
# #         os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)
# # print('Успешно созданы папки val и train с классами швов!')

# train_dir = r"data\binary_root\learning\train"
# val_dir = r"data\binary_root\learning\val"
# good_weld_dir = r"data\binary_root\welds\good_weld"
# bad_weld_dir = r"data\binary_root\welds\bad_weld"

# classes = ['bad_weld','good_weld']

# good_weld_files = [f for f in os.listdir(good_weld_dir) if os.path.isfile(os.path.join(good_weld_dir, f))]
# bad_weld_files = [f for f in os.listdir(bad_weld_dir) if os.path.isfile(os.path.join(bad_weld_dir, f))]

# # Перемещение изображений в папки train и val

# for i, file_name in enumerate(tqdm(good_weld_files)):
#     src_path = os.path.join(good_weld_dir, file_name)
#     if i % 6 == 0:
#         dst_path = os.path.join(val_dir, 'good_weld', file_name)
#     else:
#         dst_path = os.path.join(train_dir, 'good_weld', file_name)
#     shutil.copy(src_path, dst_path)

# for i, file_name in enumerate(tqdm(bad_weld_files)):
#     src_path = os.path.join(bad_weld_dir, file_name)
#     if i % 6 == 0:
#         dst_path = os.path.join(val_dir, 'bad_weld', file_name)
#     else:
#         dst_path = os.path.join(train_dir,'bad_weld', file_name)
#     shutil.copy(src_path, dst_path)

Проверяем корректность разбивки файлов:

In [ ]:
# lst_train = []
# lst_val = []
# for type in classes:
#     items_train = os.listdir(os.path.join(train_dir,type))
#     items_val =  os.listdir(os.path.join(val_dir,type))
#     lst_train.append(len(items_train))
#     lst_val.append(len(items_val))
# print('Всего изображений в деле: ', sum(lst_train) + sum(lst_val))
# print('Всего на обучении: ', sum(lst_train))
# print('Всего на валидации: ', sum(lst_val))
# print('Отношение обучающих данных к валидационным: ', sum(lst_train) / sum(lst_val))

# Трансформация и аугментация

В качестве baseline'а применяется архитектура VGG-19. У данной архитектуры есть требования к исходным изображениям:

По этой причине трансформация валидационного изображения будет выглядеть следующим образом:

In [5]:
train_dir = r"data\binary_root\learning\train"
val_dir = r"data\binary_root\learning\val"

transforms_all = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transforms_all)
val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=transforms_all)
print('Обучающих изображений: ', len(train_dataset))
print('Валидационных изображений: ', len(val_dataset))

Обучающих изображений:  29519
Валидационных изображений:  5905


# Определяю размер batches

При большом числе будет меньше обновлений весов - модель попробует учесть все изображения в одном разе. При малом числе батчей - веса будут обновляться постоянно, но модель будет учитывать малое число изображений. Начнём с 8, буду исследовать для:

In [6]:
batch_size = 2# размер бача, т.е. кол-во пикчей, используемых на 1 эпохе/

Применение трансформаций для изображений:

In [7]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=2) 


# Определение архитектуры модели:

Архитектура исследуемой модели представлена ниже:

In [8]:
# Загрузка предобученных моделей
model = models.vgg11(pretrained=True)
#model = models.vgg19(pretrained=True)
#alexnet = models.alexnet(pretrained=True)
#model = models.densenet121(pretrained=True) 
#model = models.resnet18(pretrained=True)
#model = models.resnet50(pretrained=True)

# Замена последнего полносвязного слоя на новый слой для 6 классов
# num_classes = 2
# model.classifier._modules['2'] = torch.nn.Linear(4096, num_classes)
# Замена последнего полносвязного слоя на новый слой для 6 классов ДЛЯ RESNET
# num_classes = 6
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_classes)
# Вывод архитектуры модели с новым слоем
# Загрузка предобученной модели AlexNet
#model = models.alexnet(pretrained=True)
# weights= VGG19_Weights.DEFAULT
# Замена последнего слоя классификатора
num_features = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(num_features, 1)  # Заменяем на один выходной нейрон

# Применение функции активации (например, сигмоиды) для бинарной классификации
model.classifier.add_module("sigmoid", nn.Sigmoid())
# Печать измененной модели
print(model)

C:\Users\HP\anaconda3\envs\gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\anaconda3\envs\gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [9]:
device = torch.device("cuda:0")
model = model.to(device)

Ниже определяем лосс-функцию. По дефолту это Кросс-энтропия, т.к в примере была именно она

In [10]:
criterion = torch.nn.BCELoss() # функция потерь

Learning_rate = 0.001 по дефолту

In [11]:
learning_rate = 0.001

Оптимайзер в начале исследования всегда лучше начинать с СГД:

In [12]:
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# beta1 = 0.9
# beta2 = 0.999
# eps = 0.00001
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1, beta2), eps=eps, amsgrad = True)
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Rmspror(model.parameters(), lr=learning_rate)
# alpha = 0.01
# optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=alpha)

Хочу отметить, что на этом этапе я усвоил урок - параметры оптимизатора - очень важны. Поменяв alpha для RMprop с 0.9 до 0.1 результат изменился в 3,5 раза.

Уменьшение шага градиента сначала поставим 7

In [13]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # уменьшение шага градиентного спуска

Число эпох 15, посмотрим как долго будет обучаться модель:

In [14]:
num_epochs=50


# Обучение модели:

In [15]:
best_accuracy = 0
no_improvement_count = 0
patience = 2
for epoch in range(50):
    model.train()
    for inputs, labels in tqdm(train_dataloader):
        labels = labels.unsqueeze(1).float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Оценка точности на валидационных данных
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for inputs, labels in tqdm(val_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            predicted = torch.where(outputs < 0.5, torch.tensor(0.0), torch.tensor(1.0))
#             predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted.squeeze().long() == labels).sum().item()
            accuracy = correct / total
    
    # Проверяем, улучшилась ли точность на этой эпохе
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        no_improvement_count = 0
        print('Модель улучшена!')
    else:
        no_improvement_count += 1
        print('Модель не улучшилась, обучение продолжается')
    
    # Если точность не улучшалась в течение patience эпох, прекращаем обучение
    if no_improvement_count == patience:
        print('No improvement for {} epochs. Stopping training.'.format(patience))
        break
    
    # Выводим информацию о текущей эпохе
    print('Epoch {}/{}:'.format(epoch + 1, num_epochs), flush=True)
    print('Training Loss: {:.4f}'.format(loss.item()), flush=True)
    print('Validation Accuracy: {:.4f}'.format(accuracy), flush=True)

  0%|          | 0/14760 [00:09<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 2.00 GiB total capacity; 1.49 GiB already allocated; 0 bytes free; 1.57 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.cuda.memory_summary(device="cuda:0", abbreviated=False)

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=0

In [ ]:
torch.cuda.empty_cache()

# Сохранение обученной модели:

In [ ]:
torch.save(best_model, 'vgg13_sgd_binary.pth')